In [2]:
  from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import random

from keras.callbacks import History 
history = History()

BATCH_SIZE = 5

number_of_attributes = 35

print(tf.__version__)

2.8.2


In [4]:
df = pd.read_csv('/content/drive/MyDrive/network-traffic-classification-main/csv-files/complete_df_with_36_attrs.csv')
display(df)
attributes_array = ['10','15','17','22','24','29','31','32','33','34','36','37','38','45','46','47','48','59','60','61','62','63','64','75','76','77','78','79','80','81','82','83','84','87','88','89','90','91','92']                    

,target,10,15,17,22,24,29,31,32,33,...,81,82,83,84,87,88,89,90,91,92
0,0,60,373,40,359,14,20,5,5,4,...,319,140,319,140,6900,61440,512,8192,0,0
1,0,60,373,40,359,14,20,5,5,4,...,319,141,319,141,6900,61440,512,8192,0,0
2,0,60,376,40,362,14,20,5,5,4,...,322,141,322,141,6900,61440,512,8192,0,0
3,0,60,373,40,359,14,20,5,5,4,...,319,142,319,142,6900,61440,512,8192,0,0
4,0,60,379,40,365,14,20,5,5,4,...,325,141,325,141,6900,61440,512,8192,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377521,0,60,590,40,576,14,20,9,11,8,...,259,536,259,202,65535,61440,64463,8192,0,0
377522,0,60,590,40,576,14,20,24,38,23,...,263,536,263,151,65535,61440,64312,8192,0,0
377523,0,60,590,40,576,14,20,23,36,22,...,263,536,263,226,65535,61440,64463,8192,0,0
377524,0,60,590,40,576,14,20,25,40,24,...,263,536,263,501,65535,61440,64463,8192,0,0


In [17]:
df_train = pd.DataFrame(columns= df.columns)
df_test = pd.DataFrame(columns= df.columns)
for cur_df in df.groupby('target'):
  #print("cur_df[0]: ",cur_df[0])
  #print("cur_df[1]: ",cur_df[1].index)
  lista = list(cur_df[1].index)
  samp = random.sample(lista,k=len(lista))
  #print("obtêm k amostras da lista passada:\n", samp)
  samp_a = samp[:len(lista)//2]
  samp_b = samp[len(lista)//2:]
  #print("primeira metade é treino:\n",samp_a)
  #print("segunda metade é teste:\n",samp_b)
  train= random.choices(samp_a,k=2000)
  test= random.choices(samp_b,k=2000)
  #print("gera k copias do treino:\n",train)
  #print("gera k copias do teste:\n",test)
  df_train = pd.concat([df_train, cur_df[1].loc[train]])
  df_test = pd.concat([df_test, cur_df[1].loc[test]])

target_train = df_train.pop('target')
target_test = df_test.pop('target')
display(df_test)
df_train = np.asarray(df_train).astype(np.float32)
target_train = np.asarray(target_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
target_test = np.asarray(target_test).astype(np.float32)


,10,15,17,22,24,29,31,32,33,34,...,81,82,83,84,87,88,89,90,91,92
301880,60,543,40,529,14,20,5,6,4,6,...,489,158,489,158,16560,8760,16384,6432,0,0
45957,60,590,40,576,14,20,7,6,6,6,...,285,536,285,232,65535,61440,64767,8192,0,0
112189,60,590,40,576,14,20,16,28,15,28,...,401,536,401,203,64860,61440,63788,8192,0,0
267386,66,449,52,435,14,14,6,4,5,4,...,383,201,383,201,6432,6432,5840,6432,0,0
300333,60,590,40,576,14,20,8,9,7,9,...,291,536,291,435,9660,61440,8588,8192,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293798,60,1434,40,1420,14,20,58,94,57,94,...,1380,1380,48,48,16560,65535,15408,64871,0,0
354015,60,1434,40,1420,14,20,58,95,57,95,...,1380,1380,48,48,16560,65535,15408,64871,0,0
35786,60,1208,40,1194,14,20,6,5,5,5,...,1154,0,4,0,16560,64240,16384,63082,0,0
35786,60,1208,40,1194,14,20,6,5,5,5,...,1154,0,4,0,16560,64240,16384,63082,0,0


In [18]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(100, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [30]:
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)

# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

# Execute Trainning and Testing
#model = get_basic_model()
print('Starting training for',number_of_attributes,'attributes')
model = get_basic_model()
history = model.fit(df_train, target_train, epochs=BATCH_SIZE, verbose=5, batch_size=BATCH_SIZE, callbacks=[history])
train_accuracy = history.history['accuracy']

Starting training for 35 attributes
Epoch 1/10
4800/4800 [==============================] - 13s 3ms/step - loss: 0.7187 - accuracy: 0.7811
Epoch 2/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.3892 - accuracy: 0.8760
Epoch 3/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.3051 - accuracy: 0.9041
Epoch 4/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.2601 - accuracy: 0.9187
Epoch 5/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.2320 - accuracy: 0.9259
Epoch 6/10
1521/4800 [========>.....................] - ETA: 5s - loss: 0.2248 - accuracy: 0.9270

KeyboardInterrupt: ignored

In [27]:
print("test")
test_acc =  model.evaluate(df_test,  target_test, verbose=2, batch_size=BATCH_SIZE)

tf_predictions_probabilities = model.predict(df_test)
tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)  
  j_max = x.argmax()
  tf_predictions.append(j_max)  

conf_m = tf.math.confusion_matrix(target_test,tf_predictions)
print(conf_m)

test


KeyboardInterrupt: ignored

In [23]:
# Feature Importance using crescent connection weights

# #layer0_weights = model.layers[0].get_weights()[0] #weight
# #layer0_biases  = model.layers[0].get_weights()[1] #bias
layer1_weights = model.layers[1].get_weights()[0] #weight
# #layer1_biases  = model.layers[1].get_weights()[1] #bias
print('Layer 1 weights:',layer1_weights)
# # print('\n')

weight_attribute = []
for attr_i, weights in enumerate(layer1_weights):
  weight_attribute.append((weights.sum(), attributes_array[attr_i]))

weight_attribute = sorted(weight_attribute)
print('\n',weight_attribute)

attributes = [x for _, x in weight_attribute]
print('\n',attributes,'\n')

# Date for report file
current_date_and_time = datetime.date.today()
current_date_and_time_string = str(current_date_and_time)
# Report file content
attr_report = {
    "datetime": current_date_and_time_string,
    "sorted_attributes_by_weights": attributes
}
# Writes into the file
attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/%s"%current_date_and_time_string
with open(attr_title, "w") as f:
  json.dump(attr_report , f)
# with open("/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/last-run", "w") as f:
#   json.dump(attr_report , f)
with open(attr_title, "r") as f:
  rep = json.load(f)
  display(rep)  


Layer 1 weights: [[ 0.05638021  0.15618093 -0.21595795 ... -0.2614063  -0.08428157
   0.14752999]
 [ 0.06637266 -0.06783355  0.43500766 ...  0.09356992  0.27321416
   0.1352775 ]
 [ 0.13269615  0.10666906 -0.14300178 ... -0.3447415  -0.04945166
   0.06643742]
 ...
 [-0.6256501  -0.01250606  0.18209474 ... -0.93851966 -0.1392682
   0.12314226]
 [ 0.28839913 -0.36575347  0.05893486 ...  0.07484037 -0.42716637
  -0.32029507]
 [-0.02349812 -0.37652177  0.02529136 ... -0.35286245 -0.0066088
   0.04296464]]

 [(-18.922956, '59'), (-13.87425, '60'), (-13.74652, '80'), (-9.281849, '79'), (-7.125028, '37'), (-6.736804, '24'), (-5.5739155, '84'), (-4.23006, '38'), (-3.4349673, '87'), (-3.0888388, '78'), (-2.9273, '83'), (-1.8214712, '45'), (-1.6265583, '88'), (-1.4056556, '17'), (-1.2737694, '10'), (-0.95540845, '62'), (-0.78484946, '82'), (-0.7076217, '36'), (-0.6049819, '31'), (-0.3782741, '33'), (-0.34098536, '48'), (-0.30597627, '61'), (-0.1786784, '47'), (-0.09559961, '76'), (-0.032404184, 

{'datetime': '2022-08-25',
 'sorted_attributes_by_weights': ['59',
  '60',
  '80',
  '79',
  '37',
  '24',
  '84',
  '38',
  '87',
  '78',
  '83',
  '45',
  '88',
  '17',
  '10',
  '62',
  '82',
  '36',
  '31',
  '33',
  '48',
  '61',
  '47',
  '76',
  '32',
  '81',
  '34',
  '29',
  '46',
  '22',
  '15',
  '64',
  '77',
  '63',
  '75']}